# 🕳️ Pothole Segmentation Model Training
## DSConv + SimAM + GELU Enhanced YOLOv11

Bu notebook, çukur segmentasyon modelini Google Colab üzerinde eğitmek için tasarlanmıştır.

**Özellikler:**
- 🐍 Dynamic Snake Convolution (DSConv)
- 👁️ Simple Attention Module (SimAM)
- ⚡ GELU Aktivasyon Fonksiyonu


## 1. GPU Kontrolü ve Kurulum


In [ ]:
# GPU kontrolü
!nvidia-smi


In [ ]:
# Google Drive bağlantısı
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# GitHub'dan projeyi çek
# GITHUB_REPO değişkenini kendi repo URL'nizle değiştirin
GITHUB_REPO = "https://github.com/ErenPervan/Yolo11Model.git"

!git clone {GITHUB_REPO} /content/Yolo11Model
%cd /content/Yolo11Model


In [ ]:
# Bağımlılıkları kur
!pip install -q ultralytics>=8.3.0
!pip install -q -r requirements.txt


## 2. Veri Seti Hazırlığı


In [ ]:
# Veri setinizi Google Drive'dan veya başka bir kaynaktan yükleyin
# Drive'daki veri seti yolu - KENDİ YOLUNUZU GİRİN
DRIVE_DATASET_PATH = "/content/drive/MyDrive/datasets/pothole_dataset"

# Sembolik link oluştur
!ln -sf {DRIVE_DATASET_PATH} /content/Yolo11Model/data/pothole_dataset

# Veri seti yapısını kontrol et
!ls -la /content/Yolo11Model/data/


In [ ]:
# data/pothole.yaml dosyasını Colab için oluştur
YAML_CONTENT = '''
# Pothole Dataset - Colab
path: /content/Yolo11Model/data/pothole_dataset
train: images/train
val: images/val

names:
  0: pothole

nc: 1
'''

with open('/content/Yolo11Model/data/pothole_colab.yaml', 'w') as f:
    f.write(YAML_CONTENT)

print("✅ pothole_colab.yaml oluşturuldu")


## 3. Model Hazırlığı


In [ ]:
import sys
sys.path.insert(0, '/content/Yolo11Model')

# Custom modülleri import et
from ultralytics_custom.nn.modules import DSConv, DySnakeConv, SimAM, ConvGELU, C3k2_DSConv
from ultralytics_custom.model_builder import (
    register_custom_modules,
    PotholeSegmentationModel,
    create_pothole_model
)

# Custom modülleri kaydet
register_custom_modules()
print("✅ Custom modüller yüklendi")


In [ ]:
# Model oluştur - DSConv + SimAM + GELU ile
model = create_pothole_model(
    use_pretrained=True,
    modify_activations=True
)
print("\n✅ Model hazır - DSConv + SimAM + GELU entegre edildi")


## 4. Eğitim


In [ ]:
# Eğitim parametreleri
EPOCHS = 100
BATCH_SIZE = 16  # GPU belleğine göre ayarlayın (T4 için 16, V100 için 32)
IMG_SIZE = 640
LR = 0.01

# Eğitimi başlat
results = model.train(
    data='/content/Yolo11Model/data/pothole_colab.yaml',
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    lr0=LR,
    device=0,  # GPU
    project='/content/Yolo11Model/runs',
    name='pothole_seg_dsconv',
    exist_ok=True,
    pretrained=True,
    optimizer='SGD',
    verbose=True,
    # Augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
)


## 5. Sonuçları Görüntüle


In [ ]:
# Eğitim sonuçlarını görüntüle
from IPython.display import Image, display
import os

results_dir = '/content/Yolo11Model/runs/pothole_seg_dsconv'

# Confusion Matrix
if os.path.exists(f'{results_dir}/confusion_matrix.png'):
    print("📊 Confusion Matrix:")
    display(Image(filename=f'{results_dir}/confusion_matrix.png'))

# Results
if os.path.exists(f'{results_dir}/results.png'):
    print("📈 Training Results:")
    display(Image(filename=f'{results_dir}/results.png'))


In [ ]:
# Validation sonuçları
val_results = model.val()
print(f"\n📊 Validation Sonuçları:")
print(f"  mAP50: {val_results.box.map50:.4f}")
print(f"  mAP50-95: {val_results.box.map:.4f}")


## 6. Model Kaydetme ve Export


In [ ]:
# En iyi modeli Google Drive'a kopyala
import shutil

# Drive'da models klasörü oluştur
!mkdir -p /content/drive/MyDrive/models

# Best model'i kopyala
shutil.copy(
    '/content/Yolo11Model/runs/pothole_seg_dsconv/weights/best.pt',
    '/content/drive/MyDrive/models/pothole_seg_dsconv_best.pt'
)
print("✅ Model Google Drive'a kaydedildi: /content/drive/MyDrive/models/pothole_seg_dsconv_best.pt")


## 7. Test Tahmini


In [ ]:
# Test görüntüsü üzerinde tahmin
# TEST_IMAGE yolunu kendi görüntünüzle değiştirin
TEST_IMAGE = '/content/Yolo11Model/data/pothole_dataset/images/val/'  

import glob
test_images = glob.glob(TEST_IMAGE + '*.jpg')[:3]  # İlk 3 görüntü

results = model.predict(
    source=test_images,
    save=True,
    project='/content/Yolo11Model/runs/predict',
    name='pothole_test',
    conf=0.25  # Confidence threshold
)

# Sonuçları görüntüle
from PIL import Image as PILImage
for i, r in enumerate(results):
    im_array = r.plot()
    im = PILImage.fromarray(im_array[..., ::-1])
    print(f"\n🖼️ Görüntü {i+1}:")
    display(im)
